In [40]:
from mdp import TradeExecutionEnv, DiscreteTradeSizeWrapper

SEED = 42
HORIZON = 5 * 12 * 8
UNITS_TO_SELL = 240

env = TradeExecutionEnv()

trade_sizes = {
  0: 0,
  1: 1,
  2: 2,
  3: 4,
  4: 8,
  5: 16,
  6: 32,
  7: 64,
  8: 128,
  9: 240
}
env = DiscreteTradeSizeWrapper(env, trade_sizes)

In [41]:
env.reset(UNITS_TO_SELL, HORIZON, SEED)

{'open': 860    0.514604
 861    0.520626
 862    0.514098
 863    0.501656
 864    0.511593
 865    0.497741
 Name: Open, dtype: float64,
 'high': 860    0.523637
 861    0.526046
 862    0.521529
 863    0.512797
 864    0.516411
 865    0.498946
 Name: High, dtype: float64,
 'low': 860    0.511593
 861    0.514001
 862    0.501355
 863    0.497441
 864    0.495333
 865    0.479193
 Name: Low, dtype: float64,
 'close': 860    0.520253
 861    0.514604
 862    0.501957
 863    0.511292
 864    0.497531
 865    0.483890
 Name: Close, dtype: float64,
 'volume': 860   -0.942510
 861   -0.953533
 862   -0.936444
 863   -0.937798
 864   -0.940946
 865   -0.920178
 Name: Volume, dtype: float64,
 'units_sold': 0.0,
 'cost_basis': 0}

In [47]:
obs, r, done, _, info = env.step(8)

In [49]:
r

-10

In [6]:
obs["low"].to_numpy()

array([0.51400147, 0.50135522, 0.49744056, 0.49533258, 0.47919251,
       0.47003906])

In [7]:
import torch

In [20]:
data = torch.FloatTensor([
  obs["low"].to_numpy(),
  obs["high"].to_numpy(),
  obs["close"].to_numpy(),
  obs["open"].to_numpy(),
  obs["volume"].to_numpy(),
]).T

In [22]:
torch.concat([
  data,
  torch.repeat_interleave(torch.FloatTensor([[obs["units_sold"]]]), 6, 0),
  torch.repeat_interleave(torch.FloatTensor([[obs["cost_basis"]]]), 6, 0),
], dim=1)

tensor([[ 0.5140,  0.5260,  0.5146,  0.5206, -0.9535,  0.0667,  0.4574],
        [ 0.5014,  0.5215,  0.5020,  0.5141, -0.9364,  0.0667,  0.4574],
        [ 0.4974,  0.5128,  0.5113,  0.5017, -0.9378,  0.0667,  0.4574],
        [ 0.4953,  0.5164,  0.4975,  0.5116, -0.9409,  0.0667,  0.4574],
        [ 0.4792,  0.4989,  0.4839,  0.4977, -0.9202,  0.0667,  0.4574],
        [ 0.4700,  0.4965,  0.4712,  0.4833, -0.8769,  0.0667,  0.4574]])

In [17]:
torch.repeat_interleave(torch.FloatTensor([obs["units_sold"]]), 6)

tensor([0.0667, 0.0667, 0.0667, 0.0667, 0.0667, 0.0667])

In [19]:
data.shape

AttributeError: 'torch.Size' object has no attribute 'shape'

In [30]:
import pandas as pd
df = pd.read_csv("./data/5m_intraday_data.csv")

In [31]:
df[df['Open'].isna()]

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
1462,1669398600,0,2022-11-25 17:50:00,NaN,NaN,NaN,NaN,NaN
1463,1669398900,0,2022-11-25 17:55:00,NaN,NaN,NaN,NaN,NaN


In [39]:
df.iloc[1462]

Timestamp             1669398600
Gmtoffset                      0
Datetime     2022-11-25 17:50:00
Open                         NaN
High                         NaN
Low                          NaN
Close                        NaN
Volume                       NaN
Name: 1462, dtype: object

In [38]:
df.interpolate().iloc[1462]

Timestamp             1669398600
Gmtoffset                      0
Datetime     2022-11-25 17:50:00
Open                  147.989995
High                  148.030003
Low                   147.886667
Close                 147.981669
Volume            2406348.333333
Name: 1462, dtype: object